In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from day20 import Gate, Inverter

# G = nx.Graph()
# G.add_nodes_from([1,2,3])
# G.add_edges_from([(1,2), (2,3), (3,1)])
# nx.draw(G, with_labels=True)
# plt.show()

Gate.load("./input20.txt")

color_map = []
for inv in [gate for gate in Gate.gates.values() if isinstance(gate, Inverter)]:
    # print(inv, [g for g in inv.inputs.keys()], [g for g in inv.outputs])
    print(len(inv.inputs), inv, len(inv.outputs))

for gate in Gate.G:
    color_map.append(gate.color)

# G = nx.DiGraph()

# lines = [line.strip() for line in open("./input20.txt", "r")]

# for line in lines:
#     name, dest = None, None
#     if line.startswith("broadcaster"):
#         name = "*"
#         dest = [s.strip() for s in line.strip()[15:].split(",")]
#     else:
#         source, sinks = line.strip().split("->")
#         name = source[1:].strip()
#         dest = [s.strip() for s in sinks.split(",")]
# for gate in Gate.gates.values():
#     G.add_node(gate)
#     G.add_edges_from(list(zip([gate]*len(gate.outputs), gate.outputs)))

# for gate in Gate.G:
#     print(gate, type(gate))

# color_map = ['red' if node.startswith("%") else 'green' for node in G]
# color_map = [gate.color for gate in G]
#, k=3/math.sqrt(Gate.G.order())
pos = nx.spring_layout(Gate.G, pos={Gate.broadcaster: (0,0), Gate.gates["rr"]:(20,20), Gate.gates["sl"]:(50,40), Gate.gates["jz"]:(20,-20), Gate.gates["pq"]:(50,-40), Gate.gates["rx"]:(80,0)}, fixed=[Gate.broadcaster,Gate.gates["rr"],Gate.gates["sl"],Gate.gates["jz"],Gate.gates["pq"],Gate.gates["rx"]])
nx.draw_networkx(Gate.G, pos, with_labels=True, node_color=color_map)
print(Gate.G)
plt.show()

In [ ]:
# jz: Gate = Gate.gates["jz"]
ft: Gate = Gate.gates["ft"]

def related(source:Gate, nodes:set[Gate]=None) -> list[Gate]:
    if nodes is None:
        nodes = set([Gate.broadcaster, ft])
    nodes.add(source)
    for g in source.outputs:
        if g not in nodes:
            related(g, nodes)
    for g in source.inputs:
        if g not in nodes:
            related(g, nodes)
    return nodes

relations = {Gate.gates[node]: related(Gate.gates[node]) for node in ["jz", "sl", "pq", "rr"]}

# def graph_relation(r: dict[Gate: list[Gate]]):

for source, nodes in relations.items():
    g = nx.DiGraph()
    g.add_nodes_from(nodes)
    for n in nodes:
        outputs = [z for z in n.outputs if z in nodes]
        g.add_edges_from(zip([n]*len(outputs), outputs))
    pos = nx.spring_layout(g, pos={Gate.broadcaster: (0,0), source:(70,70), Gate.gates["ft"]:(80,80)}, fixed=[Gate.broadcaster,source,Gate.gates["ft"]])
    nx.draw_networkx(g, pos=pos, with_labels=True, node_color=[gate.color for gate in g])
    plt.show()
    

In [ ]:
# Gate.load("./input20.txt")

def run_subset(name):
    inv = Gate.gates[name]
    nodes = related(inv)
    for gate in nodes:
        gate.outputs = [g for g in gate.outputs if g in nodes]
        # gate.inputs = [g for g in gate.inputs if g in nodes]


    invstates: set[str] = set()
    pulse = 0
    while True:
        pulse += 1

        Gate.broadcaster.run()
        while not Gate.runq.empty():
            gate = Gate.runq.get()
            if gate.run():
                if gate.name == "ft":
                    return pulse
        v = inv.invalue()
        # if v == "1" * len(inv.inputs):
        #     print(pulse, v, len(invstates))
        if v not in invstates:
            invstates.add(v)
        elif len(invstates) == (2**len(inv.inputs))-1:
            print("And I Loop")
            print(pulse, v, len(invstates), len(v))
            return

run_subset("jz")